In [ ]:
"""
Title : Netflix Data Wrangling Project
Name: Rodney Roy Gitonga
Cybershujaa ID :CS-DA03-26025
Program:DA1-2026
Date:21/01/2026

Description: This project focuses on cleaning, structuring and validating the Netflix Movies and TV shows Dataset


"""
import pandas as pd
import numpy as np
import datetime as dt

# 1. LOAD DATA
# Load the dataset from the specific Kaggle directory
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')


# 2. DISCOVERY

print("--- DISCOVERY ---")
print("Shape of the dataset:", df.shape)
print("\nMissing values per column:\n", df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())


# 3. STRUCTURING

print("\n--- STRUCTURING ---")

# Convert 'date_added' to datetime format
# errors='coerce' turns invalid dates into NaT (Not a Time) to prevent crashes
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(), format='mixed', errors='coerce')

# Separate 'duration' into value and unit
# We use regex to find the numbers (\d+) and the text (\w+)
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')

# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'])

print("Duration columns created successfully.")


# 4. CLEANING

print("\n--- CLEANING ---")

# Drop exact duplicates
df = df.drop_duplicates()

# Drop the 'description' column as per instructions
df = df.drop(columns=['description'])

# --- Impute Missing Directors (Optimized) ---
# Logic: If a Director+Cast combo appears 3+ times, use that to fill missing directors.
# Create a temporary column for the pair
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts()
# Filter for pairs that appear 3 or more times
frequent_pairs = counts[counts >= 3].index

# Create a dictionary for mapping: {Cast_String : Director_Name}
director_map = {}
for pair in frequent_pairs:
    if isinstance(pair, str):
        parts = pair.split('---')
        if len(parts) == 2:
            director_map[parts[1]] = parts[0]

# Fill missing directors using the map
df['director'] = df['director'].fillna(df['cast'].map(director_map))
# Fill remaining missing directors with 'Not Given'
df['director'] = df['director'].fillna('Not Given')


# --- Impute Missing Countries (Optimized) ---
# Logic: Use the Director to find the Country
# Create a mapping dictionary: {Director_Name : Country_Name}
# We drop rows where director or country is NaN to build a clean reference map
clean_subset = df.dropna(subset=['director', 'country'])
country_map = dict(zip(clean_subset['director'], clean_subset['country']))

# Fill missing countries using the director map
df['country'] = df['country'].fillna(df['director'].map(country_map))
# Fill remaining missing countries with 'Not Given'
df['country'] = df['country'].fillna('Not Given')

# Fill missing cast with 'Not Given'
df['cast'] = df['cast'].fillna('Not Given')

# Drop rows where specific crucial fields are still null
df.dropna(subset=['date_added', 'rating', 'duration'], inplace=True)

print("Missing values after cleaning:\n", df.isnull().sum())


# 5. ERROR HANDLING & VALIDATION

print("\n--- VALIDATION ---")

# Check for logical error: Date Added cannot be before Release Year
# We extract the year from date_added
df['added_year'] = df['date_added'].dt.year

# Count inconsistent records
inconsistent_dates = df[df['added_year'] < df['release_year']]
print(f"Records where date_added < release_year: {len(inconsistent_dates)}")

# (Optional) Fix or Drop inconsistent dates.
# For this assignment, we will drop them to ensure data logical accuracy.
df = df.drop(inconsistent_dates.index)

# Remove temporary columns used for wrangling
cols_to_drop = ['dir_cast', 'added_year']
# Only drop if they exist
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

# Final check
print("\nFinal shape:", df.shape)
print("Final Missing values:\n", df.isnull().sum())


# 6. EXPORT / PUBLISH

# Save to the working directory in Kaggle
df.to_csv('cleaned_netflix.csv', index=False)
print("\nFile 'cleaned_netflix.csv' saved successfully!")